<a href="https://colab.research.google.com/github/Kennethfargose/Time-Series-Analysis/blob/main/DARTS_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install darts


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install tensorboardX

In [ ]:
pip install sktime

In [ ]:
import pandas as pd
import pytest
import time
import pandas as pd
import numpy as np
import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries,datetime_attribute_timeseries
from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel, NHiTSModel
from darts.metrics import mape, smape, rmse,mae,ope, r2_score, rmsle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import datetime as dt
from functools import reduce

import pmdarima as pmd
import statsmodels.api as sm
from scipy.stats import normaltest
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
from darts.dataprocessing.transformers.boxcox import BoxCox
import numpy as np
from sktime.performance_metrics.forecasting import MeanSquaredScaledError
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel, Prophet, AutoARIMA
from darts.metrics import mape,mse
from darts.utils.statistics import check_seasonality, plot_acf

from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging


In [ ]:
def set_dataframe_index(df, index_column):
    '''This function is to reset the index since the date isn't recorded'''
    return df.set_index(index_column)

def multi_uni_df(df, state, store_ids):
    '''To select one or more store i.e uni or multi varaint'''
    cols = []
    for store_id in store_ids:
        cols.append(df[state + "_" + store_id])
    return pd.concat(cols, axis=1)

def get_timeseries(df, state, store_ids):
    '''This function is to store the timeseries type'''
    df_states = multi_uni_df(df, state, store_ids)
    ts_type = ""
    if len(store_ids) == 1:
        ts_type = state + "_" + store_ids[0] + "_univariate"
    else:
        ts_type = state + "_" + "".join(store_ids) + "_multivariate"
    return df_states, ts_type

def create_timeseries(df, value_cols):
    '''This function makes sure that all name of the columns is constant in this case sale to convert the dataframe to a series'''
    if len(df.columns) == 1:
        df.columns = ['sale']
    else:
        for i, col in enumerate(df.columns):
            df.rename(columns={col: f"sale"}, inplace=True)
    df.index = pd.to_datetime(df.index)
    return TimeSeries.from_dataframe(df, value_cols=value_cols)

def split_timeseries(series, split_date):
    '''To split the time series'''
    train, val = series.split_after(pd.Timestamp(split_date))
    return train, val

def get_train_on(ts_type):
    train_on_map = {
        "CA_1_univariate": "CA_store_1_univariate",
        "CA_3_univariate": "CA_store_3_univariate",
        "CA_2_univariate": "CA_store_2_univariate",
        "CA_4_univariate": "CA_store_4_univariate",
        "CA_12_multivariate": "CA_stores_1_2_multivariate",
        "CA_23_multivariate": "CA_stores_2_3_multivariate",
        "CA_34_multivariate": "CA_stores_3_4_multivariate",
        "CA_14_multivariate": "CA_stores_1_4_multivariate",
        "WI_1_univariate": "WI_stores_1_univariate",
        "WI_23_multivariate": "WI_stores_2_3_multivariate",
        "WI_31_multivariate": "WI_stores_1_3_multivariate",
        "WI_2_univariate": "WI_stores_2_univariate",
        "WI_3_univariate": "WI_stores_3_univariate",
        "TX_1_univariate": "TX_stores_1_univariate",
        "TX_23_multivariate": "TX_stores_2_3_multivariate",
        "TX_13_multivariate": "TX_stores_1_3_multivariate",
        "TX_2_univariate": "TX_stores_2_univariate",
        "TX_3_univariate": "TX_stores_3_univariate",
    }
    return train_on_map.get(ts_type, None)





In [ ]:
df_ca = set_dataframe_index(pd.read_csv(r"/content/drive/MyDrive/CA_all.csv"), 'date')
df_wi = set_dataframe_index(pd.read_csv(r"/content/drive/MyDrive/WI_all.csv"), 'date')
df_tx = set_dataframe_index(pd.read_csv(r"/content/drive/MyDrive/TX_all.csv"), 'date')

In [ ]:

df_states, ts_type = get_timeseries(df_ca, "CA", ["3"])
print(ts_type)
series = create_timeseries(df_states, value_cols="sale")
train, val = split_timeseries(series, '20150522')
trained_on = get_train_on(ts_type)
print(trained_on)

In [ ]:
def scale_timeseries(series,train,val):
    '''To scale the series and the train and val data'''
    transformer = Scaler()
    train_transformed = transformer.fit_transform(train)
    val_transformed = transformer.fit_transform(val)
    series_transformed = transformer.fit_transform(series)
    return series_transformed,train_transformed,val_transformed

In [ ]:
series_transformed, train_transformed, val_transformed = scale_timeseries(series,train,val)

In [ ]:
my_model_t = TransformerModel(
    input_chunk_length=250,
    output_chunk_length=70,
    batch_size=32,
    n_epochs=50,
    model_name="transformer",
    nr_epochs_val_period=10,
    d_model=64,
    nhead=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    dim_feedforward=128,
    dropout=0.1,
    activation="relu",
    random_state=42,
    save_checkpoints=True,
    force_reset=True,
)

In [ ]:
import time

In [ ]:
start_time=time.time()
my_model_t.fit(series=
               train_transformed, val_series=val_transformed, verbose=True)
end_time=time.time()
time_taken=end_time-start_time

In [ ]:
def get_metrics_table(timeseries_id_type, datapoints, model_dict, trained_on, val_data, metric_funcs, time_taken):
    results = []

    for name, model in model_dict.items():
        pred_series = model.predict(n=len(val_data))


        metric_values = [f(pred_series, val_data) for f in metric_funcs]
        results.append([name] + metric_values)

    metric_names = ["timeseries_id_type", "datapoints", "Trained_on", "time taken(s)", "Model"] + [f.__name__ for f in metric_funcs]
    metric_values = [timeseries_id_type, datapoints, trained_on, time_taken]
    metric_values += [result for result in results[0]]

    metrics_table = pd.DataFrame([metric_values], columns=metric_names)

    return metrics_table

In [ ]:
model_dict = { "Transfomer":my_model_t} #
metric_funcs = [smape,mape,rmse,mae]
datapoints=len(series)
timeseries_id_type=ts_type
trained_on=trained_on

metrics_table = get_metrics_table(timeseries_id_type,datapoints,model_dict,trained_on, val, metric_funcs,time_taken)
metrics_table